In [19]:
from __future__ import print_function

import os
import sys
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as dataUtil

In [76]:
BASE_DIR = '/Users/kalpeshpatel/Downloads/'
GLOVE_DIR = os.path.join(BASE_DIR, 'Glove')
MAX_SEQUENCE_LENGTH = 3
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 16

In [77]:
mat_contents = sio.loadmat('/users/kalpeshpatel/Downloads/data.mat')
xx = mat_contents['data']

In [78]:
## Extract vocabulary
yy = xx['vocab'][0,0]
index_to_word = {}
word_to_index = {}
vocab_size = yy.shape[1]
print("vocab size:" + str(vocab_size))
for i in range(vocab_size):
    word = yy[0][i][0]
    #print(word)
    index_to_word[i] = word
    word_to_index[word] = i
#print(word_to_index['just'])
#print(index_to_word[11])

vocab size:250


In [79]:
xx = mat_contents['data']
training = xx['trainData'][0,0]
training_x = training[0:3,].T
training_x = training_x -1
print("training_x:" + str(training_x.shape))
training_y = training[3,:].T
training_y = training_y -1

tiny_x = training_x[0:200]
tiny_y = training_y[0:200]
combined_data = dataUtil.TensorDataset(torch.tensor(tiny_x,dtype = torch.long),torch.tensor(tiny_y,dtype = torch.long))
dataloader = dataUtil.DataLoader(combined_data,batch_size = BATCH_SIZE,shuffle = True)
for i,input_x in enumerate(dataloader):
    input_tensor = input_x[0]
    target_tensor = input_x[1]
    print(input_tensor)


training_x:(372550, 3)
tensor([[  31,  222,  199],
        [  90,   67,   49],
        [ 189,  245,  221],
        [  25,  101,   10],
        [ 237,   37,  236],
        [  24,  138,   31],
        [  22,    5,   22],
        [  25,   89,  198],
        [  35,   64,   25],
        [  79,   22,  169],
        [ 187,   31,  192],
        [ 203,  245,   22],
        [ 241,   31,   22],
        [ 212,   55,  202],
        [  73,  227,  147],
        [ 108,  173,  202]])
tensor([[ 216,  135,   32],
        [ 157,  227,  197],
        [ 187,   90,    0],
        [  89,   99,  123],
        [ 200,  185,  246],
        [  35,  170,  123],
        [  25,   31,  192],
        [ 241,   31,  222],
        [ 161,  157,  151],
        [ 192,  136,   89],
        [ 173,   25,  205],
        [ 115,  108,  157],
        [ 161,  241,   31],
        [  12,   10,  157],
        [ 225,   34,  166],
        [ 222,   31,  157]])
tensor([[  31,  105,  192],
        [  41,  220,   24],
        [ 157,  227,  1

In [80]:
class buildModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(buildModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = nn.Linear(3*embedding_dim,128) # 3 is the size of context and 128 are number of output
        self.linear2 = nn.Linear(128, vocab_size)
    def forward(self,inputs):
        embeds = self.embeddings(inputs).view((-1,3*100))
        #embeds = self.embeddings(inputs)
        x = F.tanh(self.linear1(embeds))
        x = self.linear2(x)
        out = F.softmax(x,dim = 1)
        return(out)

In [81]:
losses = []
loss_function = nn.CrossEntropyLoss()
model = buildModel(vocab_size,EMBEDDING_DIM)
opt = optim.SGD(model.parameters(),lr = 0.01)

In [82]:
test = torch.tensor([[1,2,4],[4,3,2]])
embedding = nn.Embedding(10, 3)
xx = embedding(test)
print(xx)
xx.view(-1,3*3)


tensor([[[-0.6081,  0.2953, -0.1917],
         [-2.0608, -0.2215, -1.1001],
         [-0.8123,  1.6028,  0.6948]],

        [[-0.8123,  1.6028,  0.6948],
         [ 0.2250,  1.0011, -0.2730],
         [-2.0608, -0.2215, -1.1001]]])


tensor([[-0.6081,  0.2953, -0.1917, -2.0608, -0.2215, -1.1001, -0.8123,
          1.6028,  0.6948],
        [-0.8123,  1.6028,  0.6948,  0.2250,  1.0011, -0.2730, -2.0608,
         -0.2215, -1.1001]])

In [83]:

running_loss = 0
total_loss = torch.Tensor([0])
epoch = 1
    
for i,input_x in enumerate(dataloader):
    input_tensor = input_x[0]
    target_tensor = input_x[1]
    opt.zero_grad()
    model.zero_grad()
    log_probs = model(input_tensor)
    loss = loss_function(log_probs, target_tensor)
    loss.backward()
    opt.step()
    running_loss += loss.item()
    print("batch: running_loss: ", i, running_loss)
    

batch: running_loss:  0 5.521697044372559
batch: running_loss:  1 11.043093204498291
batch: running_loss:  2 16.56463384628296
batch: running_loss:  3 22.085818767547607
batch: running_loss:  4 27.60733127593994
batch: running_loss:  5 33.12857007980347
batch: running_loss:  6 38.64979124069214
batch: running_loss:  7 44.171300411224365
batch: running_loss:  8 49.69272041320801
batch: running_loss:  9 55.214001178741455
batch: running_loss:  10 60.73538112640381
batch: running_loss:  11 66.25717210769653
batch: running_loss:  12 71.77846193313599


In [62]:
def validation_loop(learning_rate):
    complete_hist = {}
    for lr in learning_rate:
        print(lr)
        opt = RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
        model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=['acc'])
        hist = model.fit(training_x,training_y_one,epochs=10, batch_size=32,
                         validation_data = (valid_x,valid_y_one))
        complete_hist[lr] = hist.history
    return(complete_hist)

In [16]:
#hist = model.fit(training_x,training_y_one,epochs=2, batch_size=32,validation_data = (valid_x,valid_y_one))
#print(hist.history)
return_hist = validation_loop([0.001,0.01])

0.001
Train on 372550 samples, validate on 46568 samples
Epoch 1/10
372550/372550 [==============================] - 67s 180us/step - loss: 3.2765 - acc: 0.2956 - val_loss: 3.1721 - val_acc: 0.3051
Epoch 2/10
372550/372550 [==============================] - 68s 183us/step - loss: 3.0924 - acc: 0.3245 - val_loss: 3.0921 - val_acc: 0.3221
Epoch 3/10
372550/372550 [==============================] - 65s 175us/step - loss: 3.0524 - acc: 0.3344 - val_loss: 3.0671 - val_acc: 0.3341
Epoch 4/10
372550/372550 [==============================] - 66s 177us/step - loss: 3.0220 - acc: 0.3398 - val_loss: 3.0443 - val_acc: 0.3371
Epoch 5/10
372550/372550 [==============================] - 65s 173us/step - loss: 2.9976 - acc: 0.3444 - val_loss: 3.0349 - val_acc: 0.3343
Epoch 6/10
372550/372550 [==============================] - 71s 190us/step - loss: 2.9839 - acc: 0.3470 - val_loss: 3.0160 - val_acc: 0.3439
Epoch 7/10
372550/372550 [==============================] - 65s 175us/step - loss: 2.9671 - acc: 

In [17]:
for i in return_hist.keys():
    print("lr: " + str(i) + " Validation accuracy: " + str (np.average(return_hist[i]['val_acc'])))

lr: 0.001 Validation accuracy: 0.336333963237
lr: 0.01 Validation accuracy: 0.218967960831


In [18]:
from prettytable import PrettyTable
table = PrettyTable()
accuracy = 0
table.field_names = ["#", "Word1", "word2", "word3", "expected", "actual1", "actual2","actual3"]
classes = model.predict(test_x,batch_size = 32)
for  i in range(classes.shape[0]):
    output = np.random.choice(a= yy[0],size = 3,p = classes[i,:], replace = False)
    if (index_to_word[test_y[i]] in output):
        accuracy += 1
    #table.add_row([i,index_to_word[test_x[i,0]],index_to_word[test_x[i,1]], index_to_word[test_x[i,2]],index_to_word[test_y[i]],output[0],output[1],output[2]])    
print("Accuracy: "+ str(accuracy/classes.shape[0]))
#print(table) 

Accuracy: 0.320155471568459
